In [39]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup

In [22]:
# https://www.hanbit.co.kr/academy/books/new_book_list.html?page=1&cate_cd=&srt=&searchKey=&keyWord=
# https://www.hanbit.co.kr/academy/books/book_view.html?p_code=B7192437175
# location='/academy/books/book_view.html?p_code=B7192437175'

In [40]:
base_url = 'https://hanbit.co.kr'
main_url = '/academy/books/new_book_list.html'
url = f'{base_url}{main_url}?page=1'
result = requests.get(url)
soup = BeautifulSoup(result.text)

In [41]:
lis = soup.select('li.sub_book_list')
len(lis)

20

In [42]:
li = lis[0]
href = li.select_one('.pop_quick_bg')['onclick']
href

"location='/academy/books/book_view.html?p_code=B7192437175'"

In [43]:
href[10:-1]

'/academy/books/book_view.html?p_code=B7192437175'

In [44]:
sub_url = f'{base_url}/{href}'
res = requests.get(sub_url)
book_soup = BeautifulSoup(res.text)

In [46]:
res.text

'<!DOCTYPE html>\n<html lang="ko">\n<head>\n  <meta charset="UTF-8">\n  <meta name="viewport" content="width=device-width, initial-scale=1.0">\n  <title>404 Not Found - 한빛출판네트워크</title>\n  <style>\n    html {\n      font-family: Helvetica Neue, Apple SD Gothic Neo, arial, \'나눔고딕\', Nanum Gothic, \'돋움\', Dotum, Tahoma, Geneva, sans-serif;\n    }\n    body {\n      margin: 0;\n    }\n    ol, ul {\n      list-style: none;\n      padding: 0;\n    }\n    h1 {\n      font-size: inherit;\n    }\n    a {\n      text-decoration: none;\n    }\n    .PageError {\n      text-align: center;\n      padding: 60px 0;\n    }\n    .Error_Title {\n      margin: 8px 0;\n      font-size: 60px;\n      color: #303538;\n    }\n    .Error_Description {\n      margin: 20px 0;\n      font-size: 18px;\n      line-height: 1.6em;\n      color: #525A61;\n    }\n    .Error_ButtonWrapper {\n      margin-top: 36px;\n    }\n    .Error_ButtonGroup .Error_ButtonWrapper {\n      display: inline-block;\n    }\n    .Error_But

In [56]:
lines = []
for li in lis:
    href = li.find('a')['href']
    sub_req = requests.get(base_url + href)
    sub_soup = BeautifulSoup(sub_req.text, 'html.parser')

    title = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
    info_lis = sub_soup.select('.info_list > li')
    info_dict = {}
    for li in info_lis:
        key = li.find('strong').get_text().split(' ')[0]
        val = li.find('span').get_text().strip()
        info_dict[key] = val

    author = info_dict['저자']
    try:
        translator = info_dict['번역']
    except:
        translator = '-'
    date = info_dict['출간']
    page = int(info_dict['페이지'].split(' ')[0].replace(',','')) 
    price = int(sub_soup.select_one('.pbr > strong').get_text().replace(',',''))
    lines.append([title, author, translator, date, page, price])  

In [57]:
df = pd.DataFrame(lines, columns=['제목','저자','번역','출간일','쪽수','판매가'])
df.head()

,제목,저자,번역,출간일,쪽수,판매가
0,처음 배우는 매트랩,방성완,-,2023-04-10,416,30000
1,SAS와 메타분석을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,-,2023-03-09,516,34000
2,정보교과교육론(3판),"최현종 , 전용주",-,2023-03-01,488,30000
3,"IT CookBook, 기초전자실험 with PSpice(3판)",홍순관,-,2023-02-28,308,22000
4,경제수학 강의(3판),김성현,-,2023-02-28,464,23000
